In [1]:
import numpy as np
import subprocess

Nsplit = 3                                               # number of split time segments
Nts = 6                                                  # number of timesteps of each time segment
NtimestepOffset = 4                                      # timestep offset between time segments
startTimestep = 0                                        # initial timestep of the first time segment
totalTimestep = (Nsplit-1) * NtimestepOffset + Nts       # number of timesteps for the entire time span

globalPrefix = 'AcousticMonopole'
prefixes, inputFiles, outputFiles = [], [], []
for k in range(Nsplit):
    prefixes += ['%s-%01d'%(globalPrefix,k)]
    inputFiles += ['magudi-%01d.inp'%(k)]
    outputFiles += ['%s-%01d.forward_run.txt'%(globalPrefix,k)]
print (prefixes)
print (inputFiles)
print (outputFiles)

['AcousticMonopole-0', 'AcousticMonopole-1', 'AcousticMonopole-2']
['magudi-0.inp', 'magudi-1.inp', 'magudi-2.inp']
['AcousticMonopole-0.forward_run.txt', 'AcousticMonopole-1.forward_run.txt', 'AcousticMonopole-2.forward_run.txt']


In [2]:
baseline_ic = []
icFiles = []
for k in range(Nsplit):
    baseline_ic += ['%s.ic.%1d.baseline.q'%(globalPrefix,k)]
    icFiles += [prefixes[k]+'.ic.q']
print (baseline_ic)
print (icFiles)
    
for k in range(Nsplit):
    subprocess.check_call('cp '+baseline_ic[k]+' '+icFiles[k], shell=True)
    subprocess.check_call('./forward --input '+inputFiles[k], shell=True)

['AcousticMonopole.ic.0.baseline.q', 'AcousticMonopole.ic.1.baseline.q', 'AcousticMonopole.ic.2.baseline.q']
['AcousticMonopole-0.ic.q', 'AcousticMonopole-1.ic.q', 'AcousticMonopole-2.ic.q']


In [3]:
w1 = 1.0e-7

def QoI(Nsplit_,NtimestepOffset_,outputFiles_):
    
    J = 0.0
    for k in range(Nsplit):
        fID = open(outputFiles_[k],'r')
        J += float(fID.read())
        fID.close()
        
    for k in range(Nsplit_-1):
        kOffset = startTimestep + NtimestepOffset_ * (k+1)
        matchingFile = '%s-%08d.q'%(prefixes[k],kOffset)
        diffFile = '%s.diff.q'%(prefixes[k])
        command = './qfile_zaxpy ' + diffFile + ' ' + \
                  "{:.16E}".format(-1.0) + ' ' + prefixes[k+1]+'.ic.q' + \
                  ' ' + matchingFile + ' --input ' + inputFiles[k]
        subprocess.check_call(command, shell=True)
        
        diffOutputFile = '%s.diff.txt'%(prefixes[k])
        subprocess.check_call('./spatial_inner_product '+diffFile+' '+diffFile + \
                              ' --output ' + diffOutputFile + \
                              ' --input ' + inputFiles[k], shell=True)
        fID = open(diffOutputFile,'r')
        J += 0.5 * w1 * float(fID.read())
        fID.close()
    
    return J

In [4]:
QoI0 = QoI(Nsplit,NtimestepOffset,outputFiles)
print (QoI0)

6.040377623630168e-08


In [5]:
from base import setOptionCommand

for k in range(Nsplit-1):
    commandFile = prefixes[k]+'.sh'
    fID = open(commandFile,'w')
    fID.write(setOptionCommand(inputFiles[k]))
    fID.write('setOption "enable_adjoint_restart" "true"\n')
    fID.write('setOption "number_of_timesteps" '+str(Nts-NtimestepOffset)+'\n')
    fID.write('setOption "adjoint_restart\/accumulated_timesteps" '+str(int(0))+'\n')
    fID.write('setOption "adjoint_restart\/intermediate_end_timestep" '+str(NtimestepOffset)+'\n')
    fID.write('./adjoint --input ' + inputFiles[k] + '\n')
    
    kOffset = startTimestep + NtimestepOffset * (k+1)
    matchingAdjointFile = '%s-%08d.adjoint.q'%(prefixes[k],kOffset)
    diffFile = '%s.diff.q'%(prefixes[k])
    fID.write('./qfile_zaxpy ' + matchingAdjointFile + ' ' +       \
              "{:.16E}".format(w1) + ' ' + diffFile +              \
              ' ' + matchingAdjointFile + ' --input ' + inputFiles[k] + '\n')
    
    fID.write('setOption "number_of_timesteps" '+str(NtimestepOffset)+'\n')
    fID.write('setOption "adjoint_restart\/accumulated_timesteps" '+str(Nts-NtimestepOffset)+'\n')
    fID.write('setOption "adjoint_restart\/intermediate_end_timestep" '+str(int(0))+'\n')
    fID.write('./adjoint --input ' + inputFiles[k] + '\n')
    
    fID.write('setOption "number_of_timesteps" '+str(Nts)+'\n')
    fID.close()

k = Nsplit - 1
commandFile = prefixes[k]+'.sh'
fID = open(commandFile,'w')
fID.write(setOptionCommand(inputFiles[k]))
fID.write('setOption "enable_adjoint_restart" "false"\n')
fID.write('setOption "number_of_timesteps" '+str(Nts)+'\n')
fID.write('./adjoint --input ' + inputFiles[k] + '\n')
fID.close()

for k in range(Nsplit):
    commandFile = prefixes[k]+'.sh'
    subprocess.check_call('sh '+commandFile, shell=True)

In [6]:
globalGradFile = globalPrefix + '.gradient_controlRegion.dat'
from os import path
if (path.exists(globalGradFile)):
    subprocess.check_call('rm '+globalGradFile, shell=True)

for k in range(Nsplit):
    kOffset = startTimestep + NtimestepOffset * k
    sliceGradFile = prefixes[k] + '.gradient_controlRegion.dat'
    command = './paste_control_forcing ' + globalGradFile + ' ' + sliceGradFile + ' ' + \
              str(int(totalTimestep)) + ' ' + str(int(kOffset)) + ' ' + str(int(Nts))
    subprocess.check_call(command, shell=True)
    
globalControlSpaceNorm = globalPrefix + '.norm_controlRegion.dat'
globalSensitivityFile = globalPrefix + '.adjoint_run.txt'
command = './zxdoty ' + globalSensitivityFile + ' ' +                                \
        globalGradFile + ' ' + globalGradFile + ' ' + globalControlSpaceNorm
subprocess.check_call(command, shell=True)

fID = open(globalSensitivityFile,'r')
Grad0 = float(fID.read())
fID.close()
print ('control forcing gradient: ', Grad0)

control forcing gradient:  1.344791970471636e-23


In [7]:
icGrad0 = np.zeros(Nsplit)
icGradFiles = ['']
for k in range(1,Nsplit):
    kOffset = startTimestep + NtimestepOffset * k
    adjointFile = '%s-%08d.adjoint.q' % (prefixes[k],kOffset)
    diffFile = '%s.diff.q'%(prefixes[k-1])
    icGradFiles += ['%s.ic.adjoint.q' % (prefixes[k])]
    command = './qfile_zaxpy ' + icGradFiles[k] + ' ' +                     \
              "{:.16E}".format(-w1) + ' ' + diffFile +                  \
              ' ' + adjointFile + ' --input ' + inputFiles[k]
    subprocess.check_call(command, shell=True)
    
    gradNormFile = prefixes[k] + '.spatial_inner_product.txt'
    command = './spatial_inner_product '+icGradFiles[k]+' '+icGradFiles[k] +   \
                ' --input ' + inputFiles[k] + ' --output ' + gradNormFile
    subprocess.check_call(command, shell=True)
    fID = open(gradNormFile,'r')
    icGrad0[k] = float(fID.read())
    fID.close()
print ("ic_grad0: ", icGrad0)

ic_grad0:  [0.00000000e+00 2.27163206e-14 1.24906943e-14]


In [8]:
sliceControlForcingFiles = []
for k in range(Nsplit):
    sliceControlForcingFiles += ['%s.control_forcing_controlRegion.dat'%prefixes[k]]

In [ ]:
Nk = 20
Ak = 10.0**(12.0-0.25*np.array(range(Nk)))
QoIk = np.zeros((Nk,),dtype=np.double)
Gradk = np.zeros((Nk,),dtype=np.double)
ek = np.zeros((Nk,),dtype=np.double)

In [ ]:
globalControlForcingFile = globalPrefix + '.control_forcing_controlRegion.dat'
from os import path
if (path.exists(globalControlForcingFile)):
    subprocess.check_call('rm '+globalControlForcingFile, shell=True)

outputFiles = []
for k in range(Nsplit):
    outputFiles += ['%s-%01d.forward_run.1.txt'%(globalPrefix,k)]

w2 = 1.0e-19
Grad = Grad0 + np.sum(icGrad0) * w2
print ('Grad: ', Grad)
    
for k in range(Nk):
    command = './zaxpy '+globalControlForcingFile+' '+"{:.16E}".format(Ak[k])+' '+globalGradFile
    subprocess.check_call(command, shell=True)
    for j in range(1,Nsplit):
        command = './qfile_zaxpy ' + icFiles[j] + ' ' +                                \
                  "{:.16E}".format(Ak[k]*w2) + ' ' + icGradFiles[j] +                  \
                  ' ' + baseline_ic[j] + ' --input ' + inputFiles[j]
        subprocess.check_call(command, shell=True)
    for j in range(Nsplit):
        jOffset = startTimestep + NtimestepOffset * j
        command = './slice_control_forcing ' + sliceControlForcingFiles[j] + ' ' + globalControlForcingFile + ' ' + \
                    str(int(totalTimestep)) + ' ' + str(int(jOffset)) + ' ' + str(int(Nts))
        subprocess.check_call(command, shell=True)
        
        commandFile = prefixes[j]+'.sh'
        fID = open(commandFile,'w')
        fID.write(setOptionCommand(inputFiles[j]))
        fID.write('setOption "controller_switch" "true"\n')
        fID.write('./forward --input ' + inputFiles[j] + ' --output ' + outputFiles[j] + '\n')
        fID.write('setOption "controller_switch" "false"\n')
        fID.close()
        subprocess.check_call('sh '+commandFile, shell=True)
    QoIk[k] = QoI(Nsplit,NtimestepOffset,outputFiles)
    Gradk[k] = (QoIk[k]-QoI0)/Ak[k]
    ek[k] = abs( (Gradk[k]-Grad)/Grad )
    print ("{:.16E}".format(Ak[k]), "{:.16E}".format(QoIk[k]), "{:.16E}".format(Gradk[k]), "{:.16E}".format(ek[k]))

Grad:  1.3447919708237063e-23


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(1)
plt.loglog(Ak,ek,'ok')